In [1]:
# set WORKDIR to the top of experiment repository
%cd ..
# %pwd

/data_science/projects/LOGOSAI.TECH_external/CHALLENGE/Experiment-repository-template


/data_science/projects/LOGOSAI.TECH_external/CHALLENGE/Experiment-repository-template/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import mlflow
from sklearn.metrics import matthews_corrcoef, f1_score
from src.utils import timestamp
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
from transformers.pipelines.text_classification import ClassificationFunction
import torch

from src.settings import (
    MLFLOW_TRACKING_USERNAME,
    EXPERIMENT_NAME,
    )


# Check if a GPU is available and set the device
device = 0 if torch.cuda.is_available() else -1

# select the model for evaluation
all_models = {
    'all-mpnet-base-v2': 'sentence-transformers/all-mpnet-base-v2',
    'stsb-roberta-large': 'cross-encoder/stsb-roberta-large',
    'stsb-roberta-base': 'cross-encoder/stsb-roberta-base',
    'Legal-BERT': 'nlpaueb/legal-bert-base-uncased',
    'EURLEX-BERT': 'nlpaueb/bert-base-uncased-eurlex',
    'SciBERT': 'allenai/scibert_scivocab_uncased',
    # fine-tuned models below
    #
}

selected_model = all_models['stsb-roberta-large']


In [3]:
# load test dataset
dataset_path = 'data/test_clean.parquet'
df_test = pd.read_parquet(dataset_path)

# X
sentence_pairs = list(zip(df_test['text'].tolist(),df_test['text_b'].tolist()))
# sentence pairs as list of dicts for transformer's pipeline
sentence_pairs_lods = [{"text": x[0], "text_pair": x[1]} for x in sentence_pairs]

# y_true
labels_true = df_test['label'].tolist()


In [4]:
# init mlflow experiment (use existing one)
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

# load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(selected_model)
tokenizer = AutoTokenizer.from_pretrained(selected_model)

# init mlflow experiment (use existing one)
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

# run experiment
with mlflow.start_run(experiment_id=experiment.experiment_id, log_system_metrics=True) as run:
    # set run name
    mlflow.set_tag(key='mlflow.runName',
                    value=f"Evaluation_{selected_model.split('/')[1]}_{timestamp()}")
    
    # log parameters
    mlflow.log_params({
        'PyTorch Device': torch.cuda.get_device_name(torch.cuda.current_device()),
        'Model': selected_model,
        'Dataset': dataset_path,
        'AutoModel Parameters': model,
        'Tokenizer': tokenizer,
    })

    # run pipeline for model predictions
    pipe = pipeline("text-classification", 
                    model = model, 
                    tokenizer = tokenizer, 
                    padding = True, 
                    truncation = True,
                    # max_length = 512, 
                    device = device, 
                    # function_to_apply = ClassificationFunction.SIGMOID,
                    # top_k=None,
                    )

    predictions = pipe(sentence_pairs_lods)

    # binarization for cross-encoders
    if selected_model.split('/')[0] in ['cross-encoder']:
        # for threshold in [
        #     0.33,
        #     0.53,
        #     0.66,
        #     0.75,
        #     0.85,
        # ]:
        # threshold
        threshold = 0.54
        labels_pred = [0 if x['score'] <= threshold else 1 for x in predictions]

    else:
        labels_pred = [0 if x['label'] == 'LABEL_0' else 1 for x in predictions]

    f1_score_value = f1_score(y_true=labels_true, y_pred=labels_pred, pos_label=1, average='binary')
    mlflow.log_metric("F1 Score", f1_score_value)

    matthews_corrcoef_value = matthews_corrcoef(y_true=labels_true, y_pred=labels_pred)
    mlflow.log_metric("Matthews Correlation Coefficient", matthews_corrcoef_value)

    print(f"F1 Score: {f1_score_value}\nMatthews Correlation Coefficient: {matthews_corrcoef_value}")

# end experiment
mlflow.end_run()


2024/05/31 11:18:21 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


F1 Score: 0.5100502512562815
Matthews Correlation Coefficient: 0.0021392465789891314


2024/05/31 11:19:16 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/05/31 11:19:17 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


- mlflow
- Implement a classic transformer-based classifier.
    - Split the training data into train and validation sets
    - fine-tune
    - test with test data
- Obtain quality metrics on the test set. Do include F1 score and Matthews Correlation Coefficient https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html
- 

datasets


In [5]:
# import datasets

In [6]:
# dataset = datasets.load_dataset("parquet", data_files={"train": "data/train_clean.parquet", "test": "data/test_clean.parquet"})